In [2]:
p = 761; q61 = 765; q = 6*q61+1; w = 250
Zx.<x> = ZZ[]; R.<xp> = Zx.quotient(x^p-x-1)
Fq = GF(q); Fqx.<xq> = Fq[]; Rq.<xqp> = Fqx.quotient(x^p-x-1)
F3 = GF(3); F3x.<x3> = F3[]; R3.<x3p> = F3x.quotient(x^p-x-1)

In [84]:
def ext_gcd( a , b ) :
  if a.degree() < b.degree() :  a,b = b,a  #swap
  Ta = a.parent()
  lr = [ a , b ]
  ls = [ Ta(1) , Ta(0) ]
  lt = [ Ta(0) , Ta(1) ]
  while lr[-1] != 0 :
    qq = lr[-2] // lr[-1]
    rr = lr[-2] % lr[-1]
    lr.append( rr )
    ls.append( ls[-2] - qq*ls[-1] )
    lt.append( lt[-2] - qq*lt[-1] )
  #normalize
  inv_n = (lr[-2].coefficients()[0])^-1
  return lr[-2]*inv_n,ls[-2]*inv_n,lt[-2]*inv_n


def list_pad_zero( al , len ):
  al_ref = [ al[0].parent(0) ]*(len)
  al = map( lambda x,y: y if x is None else x, al , al_ref )
  return al

def poly_in_deg( a , deg ):
  return list_pad_zero( a.list() , deg+1 )


def poly_rev( a , deg ):
  r = poly_in_deg(a,deg)
  rev_r = r[::-1]
  return rev_r


def conditional_swap( condition , a , b ) :
  if 0 == condition:
    return a , b
  else :
    return b , a


In [183]:

def divstep( delta , f , g , T_ ):
  coef_k = f[0].parent()
  kx.<x> = coef_k[] #kx = T_[0][0].parent()
  #x = kx.gen()
  M2kx = T_.parent() #MatrixSpace(kx.fraction_field(),2)   
   #MatrixSpace( kx , 2 )
  f = list_pad_zero( f , max(len(f),len(g)))
  g = list_pad_zero( g , max(len(f),len(g)))
  T0 = M2kx( (1,0,-g[0]/x,f[0]/x) )
  T1 = M2kx( (0,1,g[0]/x,-f[0]/x) )
  #conditional move
  c = 1 if delta > 0 and g[0] != 0 else 0
  rc , r1 = conditional_swap( c , [delta,f,g] , [-delta,g,f] )
  T0 , T1 = conditional_swap( c , T0 , T1 )
  # elimination
  rc[0] = 1 + rc[0]
  f = rc[1]
  g = rc[2]
  rg = [ f[0] * gi - g[0] * fi for fi,gi in zip(f,g) ]
  rg.pop(0)
  return rc[0] , rc[1] , rg , T0 , T0*T_
  

def ext_gcd0( a , b ) :
  if a.degree() < b.degree() :  a,b = b,a  #swap
  Ta = a.parent()
  x = Ta.gen()
  lr = [ a , b ]
  lf = [Ta(1),Ta(0)]
  lg = [Ta(0),Ta(1)]
  print lr,"\n",lf,"\n",lg
  while lr[-1] != 0 :
    if lr[0].degree() > lr[1].degree() : 
       lr = lr[::-1]
       lf , lg = lg , lf
       print "op0: [0,1]"
    else:
       print "op0: [1,0]"
    delta = lr[1].degree() - lr[0].degree()
    lc_f = lr[0].leading_coefficient()
    lc_g = lr[1].leading_coefficient()
    lr[1] = lr[1]*lc_f - lr[0]*lc_g*(x^delta)
    lg[0] = lg[0]*lc_f-lf[0]*lc_g*(x^delta)
    lg[1] = lg[1]*lc_f-lf[1]*lc_g*(x^delta)
    print "op1: [" , lc_f , "," , lc_g , "x^" , delta , "]"
    print lr,"\n",lf,"\n",lg , "\n"
  #normalize
  inv_n = (lr[-2].leading_coefficient())^-1
  return lr[-2]*inv_n,map(lambda x:x*inv_n, lf )



In [137]:
a2 = Fqx.random_element()
a1 = Fq.random_element() * xq + Fq.random_element()
a1,a2

(1559*xq + 3949, 3290*xq^2 + 3040*xq + 4187)

In [136]:
b2 = Fqx.random_element()
b1 = Fq.random_element() * xq + Fq.random_element()
b2 , b1

(2898*xq^2 + 4092*xq + 1754, 2118*xq + 396)

In [152]:
c1 = Fq.random_element() * xq + Fq.random_element()
d1 = Fq.random_element() * xq + Fq.random_element()
c1,d1, c1.leading_coefficient()

(1332*xq + 735, 2741*xq + 3518, 1332)

In [190]:
r1 , T0 = ext_gcd2( a2*b2 , b2*c1 )


1 [3504, 1659, 2347, 1601, 2989] [3696, 833, 24, 3710] 
[1 0]
[0 1]
0 [3696, 833, 24, 3710, 0] [3723, 655, 1369, 1398] 
[      0       1]
[3696/xq 1087/xq] 
[      0       1]
[3696/xq 1087/xq] 

1 [3696, 833, 24, 3710, 0] [3680, 3010, 4122, 0] 
[      1       0]
[ 868/xq 3696/xq] 
[                  0                   1]
[          2191/xq^2 (868*xq + 427)/xq^2] 

1 [3680, 3010, 4122, 0, 0] [2276, 3708, 3757, 0] 
[      0       1]
[3680/xq  895/xq] 
[                       2191/xq^2              (868*xq + 427)/xq^2]
[                        588/xq^3 (3680*xq^2 + 981*xq + 1112)/xq^3] 

1 [3680, 3010, 4122, 0, 0] [0, 0, 0, 0] 
[      1       0]
[2315/xq 3680/xq] 
[                        2191/xq^2               (868*xq + 427)/xq^2]
[            (3701*xq + 1479)/xq^4 (2103*xq^2 + 2994*xq + 1579)/xq^4] 

1 [3680, 3010, 4122, 0, 0] [0, 0, 0, 0] 
[      1       0]
[      0 3680/xq] 
[                        2191/xq^2               (868*xq + 427)/xq^2]
[            (2774*xq + 2385)/xq^5 (320

In [189]:
r0,r1,r2, T0 = divstep( 0 , poly_rev(a,2) , poly_rev(b,2) )
T0

TypeError: divstep() takes exactly 4 arguments (3 given)

In [188]:
ext_gcd0( a2*b2 ,b2*c1 )

[3504*xq^4 + 1659*xq^3 + 2347*xq^2 + 1601*xq + 2989, 3696*xq^3 + 833*xq^2 + 24*xq + 3710] 
[1, 0] 
[0, 1]
op0: [0,1]
op1: [ 3696 , 3504 x^ 1 ]
[3696*xq^3 + 833*xq^2 + 24*xq + 3710, 3723*xq^3 + 655*xq^2 + 1369*xq + 1398] 
[0, 1] 
[3696, 1087*xq] 

op0: [1,0]
op1: [ 3696 , 3723 x^ 0 ]
[3696*xq^3 + 833*xq^2 + 24*xq + 3710, 3680*xq^2 + 3010*xq + 4122] 
[0, 1] 
[2191, 427*xq + 868] 

op0: [0,1]
op1: [ 3680 , 3696 x^ 1 ]
[3680*xq^2 + 3010*xq + 4122, 2276*xq^2 + 3708*xq + 3757] 
[2191, 427*xq + 868] 
[588*xq, 1112*xq^2 + 981*xq + 3680] 

op0: [1,0]
op1: [ 3680 , 2276 x^ 0 ]
[3680*xq^2 + 3010*xq + 4122, 0] 
[2191, 427*xq + 868] 
[1479*xq + 3701, 1579*xq^2 + 2994*xq + 2103] 



(xq^2 + 163*xq + 2868, [1197, 952*xq + 2161])

In [94]:
a

2208*xq^2 + 2007*xq + 1458

In [95]:
b

3345*xq^2 + 3134*xq + 4569

In [97]:
a*a.parent( T0[0][0] ) + b * b.parent( T0[0][1] )

xq^2

In [123]:
def jumpdivsteps(n,t,delta,f,g):
  print "jumpdivsteps(",n,t,delta,f,g,")"
  assert t>= n and n>=0
  f,g = f.truncate(t), g.truncate(t)
  print "truncated: f=",f , "  g=", g
  kx=f.parent()
  x=kx.gen()
  M2kx=MatrixSpace(kx.fraction_field(),2)
  if n == 0: return delta,f,g,(),M2kx(1)
  if n==1:
    c = 1 if delta > 0 and g[0] != 0 else 0
    Tc, Td = conditional_swap( c , M2kx((1,0,-g[0]/x,f[0]/x)), M2kx( (0,1,g[0]/x,-f[0]/x) ) )
    delta , d2 = conditional_swap( c , delta , -delta )
    fc , f2 = conditional_swap( c , f , g )
    gc , g2 = conditional_swap( c , f[0]*g-g[0]*f , g[0]*f-f[0]*g )
    return 1+delta,fc,kx(gc/x),(Tc,),Tc
  # other cases of n
  j = n//2
  print "j=",j
  delta,f1,g1,T1,P1 = jumpdivsteps(j,j,delta,f,g)
  print "P1:\n" , P1 
  f,g = P1*vector( (f,g) )
  print "P1x(f,g): f=" , f , "  g=", g
  f,g = kx(f).truncate(t-j),kx(g).truncate(t-j)
  print "truncated: f=",f , "  g=", g

  delta,f2,g2,T2,P2 = jumpdivsteps(n-j,n-j,delta,f,g)
  print "P2:\n" , P2
  f,g = P2*vector( (f,g) )
  print "P2x(f,g): f=" , f , "  g=", g
  f,g = kx(f).truncate(t-n+1),kx(g).truncate(t-n)
  print "truncated: f=",f , "  g=", g
  return delta,f,g,T2+T1,P2*P1

def jumpgcd(R0,R1):
  d = R0.degree()
  assert d>0 and d>R1.degree()
  f,g = R0.reverse(d), R1.reverse(d-1)
  delta,f,g,T,P = jumpdivsteps(2*d-1,3*d-1,1,f,g)
  return f.reverse(delta//2)/f[0]


In [191]:
jumpgcd(a*c,b)

jumpdivsteps( 5 8 1 3152*xq^3 + 3251*xq^2 + 2890*xq + 4298 4569*xq^2 + 3134*xq + 3345 )
truncated: f= 3152*xq^3 + 3251*xq^2 + 2890*xq + 4298   g= 4569*xq^2 + 3134*xq + 3345
j= 2
jumpdivsteps( 2 2 1 3152*xq^3 + 3251*xq^2 + 2890*xq + 4298 4569*xq^2 + 3134*xq + 3345 )
truncated: f= 2890*xq + 4298   g= 3134*xq + 3345
j= 1
jumpdivsteps( 1 1 1 2890*xq + 4298 3134*xq + 3345 )
truncated: f= 4298   g= 3345
P1:
[      0       1]
[3345/xq  293/xq]
P1x(f,g): f= 3134*xq + 3345   g= 3057
truncated: f= 3345   g= 3057
jumpdivsteps( 1 1 0 3345 3057 )
truncated: f= 3345   g= 3057
P2:
[      1       0]
[1534/xq 3345/xq]
P2x(f,g): f= 3345   g= 0
truncated: f= 3345   g= 0
P1:
[                    0                     1]
[             758/xq^2 (1534*xq + 2202)/xq^2]
P1x(f,g): f= 4569*xq^2 + 3134*xq + 3345   g= 285*xq + 1727
truncated: f= 4569*xq^2 + 3134*xq + 3345   g= 285*xq + 1727
jumpdivsteps( 3 3 1 4569*xq^2 + 3134*xq + 3345 285*xq + 1727 )
truncated: f= 4569*xq^2 + 3134*xq + 3345   g= 285*xq + 1727
j=

1

In [192]:
def list_mul( l , c ):
  return [ x*c for x in l ]

def list_add( l1 , l2 ):
  return [ x+y for x,y in zip(l1,l2) ]

def list_shift(l,shift) :
  src_idx = max(-shift,0)
  dst_idx = max(shift,0)
  length = max(len(l)-abs(shift),0)
  new_l = [ l[0].parent()(0) ]* len(l)
  new_l[dst_idx:dst_idx+length] = l[src_idx:src_idx+length]
  return new_l



In [206]:
def deg1mat_mul( deg1mat , mat2 ) :
  r0 , r0_ = conditional_swap( deg1mat[0][0] , mat2[1] , mat2[0] )   # use AND
  r10_u = list_mul(mat2[0][0],deg1mat[1][0])
  r10_d = list_mul(mat2[1][0],deg1mat[1][1])
  r11_u = list_mul(mat2[0][1],deg1mat[1][0])
  r11_d = list_mul(mat2[1][1],deg1mat[1][1])
  deno_u = mat2[0][2] + deg1mat[1][2]
  deno_d = mat2[1][2] + deg1mat[1][2]
  deno = max( deno_u , deno_d )
  r10 = list_add( list_shift(r10_u,deno-deno_u) , list_shift(r10_d,deno-deno_d) )
  r11 = list_add( list_shift(r11_u,deno-deno_u) , list_shift(r11_d,deno-deno_d) )
  return [ r0 , [ r10 , r11 , deno ] ]


In [209]:
def divstep_list( delta , f , g ):
  coef_k = f[0].parent()
  T00 = [ coef_k(1) , coef_k(0) , 0 ]
  T10 = [ -g[0] , f[0] , 1 ]
  T01 = [ coef_k(0) , coef_k(1) , 0 ]
  T11 = [ g[0]  , -f[0] , 1 ]
  #conditional move
  c = 1 if delta > 0 and g[0] != 0 else 0
  rd , rd_ = conditional_swap( c , delta , -delta )
  f  , g   = conditional_swap( c , f , g )
  T0 , T0_ = conditional_swap( c , [ T00, T10 ] , [ T01 , T11 ] )
  # elimination
  rd = rd + 1
  rg = [ f[0] * gi - g[0] * fi for fi,gi in zip(f,g) ]
  rg = list_shift( rg , -1 )
  return rd , f , rg , T0


In [197]:
def extgcd_list( a , b ):
  deg_a = a.degree()
  deg_b = b.degree()
  if deg_a < deg_b :
    a , b = b , a
    deg_a , deg_b = deg_b , deg_a
  if deg_a < 0 : return a
  if deg_a == deg_b : 
    b = b*a.leading_coefficient() - a*b.leading_coefficient()
  deg_b = deg_a - 1
  delta = deg_a - deg_b
  r1 = poly_rev( a , deg_a )
  r2 = poly_rev( b , deg_b )
  lenlist = deg_a + 1
  r2 = list_pad_zero( r2 , lenlist )
  coef_k = r1[0].parent()
  T00 = [ [coef_k(0)]*lenlist , [coef_k(0)]*lenlist , 0 ]
  T01 = [ [coef_k(0)]*lenlist , [coef_k(0)]*lenlist , 0 ]
  T00[0][0] = coef_k(1)
  T01[1][0] = coef_k(1)
  T0 = [T00,T01]
  print delta , r1 , r2 , "\n", T0
  delta , r1 , r2 , T1 = divstep_list( delta , r1 , r2 )
  T0 = deg1mat_mul( T1 , T0 )
  print delta , r1 , r2 , '\n' , T1 , '\n' , T0 , '\n'
  for i in range(deg_a+deg_b-1) :
    delta , r1 , r2 , T1 = divstep_list( delta , r1 , r2 )
    T0 = deg1mat_mul( T1 , T0 )
    print r0 , r1 , r2 , '\n' , T1 , '\n' , T0 , '\n'
  #normalize
  inv = r1[0]^-1
  r1 = map( lambda x: x*inv , r1 )
  T0[0][0] = [ x*inv for x in T0[0][0] ]
  T0[0][1] = [ x*inv for x in T0[0][1] ]
  print "output:"
  print delta , r1 , r2 , '\n' , T0 , '\n'
  return r1 , T0



In [211]:
extgcd_list( a2*b2 , b2*c1 )

1 [3504, 1659, 2347, 1601, 2989] [3696, 833, 24, 3710, 0] 
[[[1, 0, 0, 0, 0], [0, 0, 0, 0, 0], 0], [[0, 0, 0, 0, 0], [1, 0, 0, 0, 0], 0]]
0 [3696, 833, 24, 3710, 0] [3723, 655, 1369, 1398, 0] 
[[0, 1, 0], [3696, 1087, 1]] 
[[[0, 0, 0, 0, 0], [1, 0, 0, 0, 0], 0], [[3696, 0, 0, 0, 0], [1087, 0, 0, 0, 0], 1]] 

1 [3696, 833, 24, 3710, 0] [3680, 3010, 4122, 0, 0] 
[[1, 0, 0], [868, 3696, 1]] 
[[[0, 0, 0, 0, 0], [1, 0, 0, 0, 0], 0], [[2191, 0, 0, 0, 0], [427, 868, 0, 0, 0], 2]] 

1 [3680, 3010, 4122, 0, 0] [2276, 3708, 3757, 0, 0] 
[[0, 1, 0], [3680, 895, 1]] 
[[[2191, 0, 0, 0, 0], [427, 868, 0, 0, 0], 2], [[588, 0, 0, 0, 0], [1112, 981, 3680, 0, 0], 3]] 

1 [3680, 3010, 4122, 0, 0] [0, 0, 0, 0, 0] 
[[1, 0, 0], [2315, 3680, 1]] 
[[[2191, 0, 0, 0, 0], [427, 868, 0, 0, 0], 2], [[1479, 3701, 0, 0, 0], [1579, 2994, 2103, 0, 0], 4]] 

1 [3680, 3010, 4122, 0, 0] [0, 0, 0, 0, 0] 
[[1, 0, 0], [0, 3680, 1]] 
[[[2191, 0, 0, 0, 0], [427, 868, 0, 0, 0], 2], [[2385, 2774, 0, 0, 0], [3105, 4111, 3205, 0,

([1, 163, 2868, 0, 0],
 [[[1197, 0, 0, 0, 0], [952, 2161, 0, 0, 0], 2],
  [[2436, 2585, 0, 0, 0], [3951, 3581, 4544, 0, 0], 7]])

In [225]:

def list_pad_zero( al , len ):
  al_ref = [ al[0].parent(0) ]*(len)
  al = map( lambda x,y: y if x is None else x, al , al_ref )
  return al

def list_pad_front(al,nlen):
  assert nlen >= len(al)
  ral = list(al)
  ral[:0] = [al[0].parent(0)]*(nlen-len(al))
  return ral


def deg1mat_mul( deg1mat , mat2 ) :
  inp_poly_len = len(mat2[0][0])
  np_len = inp_poly_len+1
  mat2_r0 = [list_pad_front(mat2[0][0],np_len),list_pad_front(mat2[0][1],np_len)]
  mat2_r1 = [list_pad_zero(mat2[1][0],np_len),list_pad_zero(mat2[1][1],np_len)]
  r0 , r0_ = conditional_swap( deg1mat[0][0][0] , mat2_r1 , mat2_r0 )   # use AND
  m1_10 = deg1mat[1][0][0]
  m1_11 = deg1mat[1][1][0]
  r10_u = list_mul(mat2[0][0],m1_10)
  r10_d = list_mul(mat2[1][0],m1_11)
  r11_u = list_mul(mat2[0][1],m1_10)
  r11_d = list_mul(mat2[1][1],m1_11)
  r10 = list_add(list_pad_front(r10_u,np_len),list_pad_zero(r10_d,np_len))
  r11 = list_add(list_pad_front(r11_u,np_len),list_pad_zero(r11_d,np_len))
  return [ r0 , [ r10 , r11 ] ]

def divstep_list( delta , f , g ):
  coef_k = f[0].parent()
  T00 = [ [coef_k(1)] , [coef_k(0)] ]
  T10 = [ [-g[0]] , [f[0]]  ]
  T01 = [ [coef_k(0)] , [coef_k(1)] ]
  T11 = [ [g[0]]  , [-f[0]] ]
  #conditional move
  c = 1 if delta > 0 and g[0] != 0 else 0
  rd , rd_ = conditional_swap( c , delta , -delta )
  f  , g   = conditional_swap( c , f , g )
  T0 , T0_ = conditional_swap( c , [ T00, T10 ] , [ T01 , T11 ] )
  # elimination
  rd = rd + 1
  rg = [ f[0] * gi - g[0] * fi for fi,gi in zip(f,g) ]
  rg = list_shift( rg , -1 )
  return rd , f , rg , T0

def extgcd_list( a , b ):
  deg_a = a.degree()
  deg_b = b.degree()
  if deg_a < deg_b :
    a , b = b , a
    deg_a , deg_b = deg_b , deg_a
  if deg_a < 0 : return a
  if deg_a == deg_b : 
    b = b*a.leading_coefficient() - a*b.leading_coefficient()
  deg_b = deg_a - 1
  delta = deg_a - deg_b
  r1 = poly_rev( a , deg_a )
  r2 = poly_rev( b , deg_b )
  lenlist = deg_a + 1
  r2 = list_pad_zero( r2 , lenlist )
  coef_k = r1[0].parent()
  print delta , r1 , r2 , "\n"
  delta , r1 , r2 , T1 = divstep_list( delta , r1 , r2 )
  T0 = T1    # T1*Identity = T1
  print delta , r1 , r2 , '\n' , T1 , '\n'
  for i in range(deg_a+deg_b-1) :
    delta , r1 , r2 , T1 = divstep_list( delta , r1 , r2 )
    T0 = deg1mat_mul( T1 , T0 )
    print delta , r1 , r2 , '\n' , T1 , '\n' , T0 , '\n'
  #normalize
  inv = r1[0]^-1
  r1 = map( lambda x: x*inv , r1 )
  T0[0][0] = list_mul( T0[0][0] , inv )
  T0[0][1] = list_mul( T0[0][1] , inv )
  print "output:"
  print delta , r1 , r2 , '\n' , T0 , '\n'
  return r1 , T0



In [227]:
extgcd_list( a2*b2 ,b2*c1 )

1 [3504, 1659, 2347, 1601, 2989] [3696, 833, 24, 3710, 0] 

0 [3696, 833, 24, 3710, 0] [3723, 655, 1369, 1398, 0] 
[[[0], [1]], [[3696], [1087]]] 

1 [3696, 833, 24, 3710, 0] [3680, 3010, 4122, 0, 0] 
[[[1], [0]], [[868], [3696]]] 
[[[0, 0], [0, 1]], [[2191, 0], [427, 868]]] 

0 [3680, 3010, 4122, 0, 0] [2276, 3708, 3757, 0, 0] 
[[[0], [1]], [[3680], [895]]] 
[[[2191, 0, 0], [427, 868, 0]], [[588, 0, 0], [1112, 981, 3680]]] 

1 [3680, 3010, 4122, 0, 0] [0, 0, 0, 0, 0] 
[[[1], [0]], [[2315], [3680]]] 
[[[0, 2191, 0, 0], [0, 427, 868, 0]], [[1479, 3701, 0, 0], [1579, 2994, 2103, 0]]] 

2 [3680, 3010, 4122, 0, 0] [0, 0, 0, 0, 0] 
[[[1], [0]], [[0], [3680]]] 
[[[0, 0, 2191, 0, 0], [0, 0, 427, 868, 0]], [[2385, 2774, 0, 0, 0], [3105, 4111, 3205, 0, 0]]] 

3 [3680, 3010, 4122, 0, 0] [0, 0, 0, 0, 0] 
[[[1], [0]], [[0], [3680]]] 
[[[0, 0, 0, 2191, 0, 0], [0, 0, 0, 427, 868, 0]], [[3399, 2527, 0, 0, 0, 0], [3992, 1135, 121, 0, 0, 0]]] 

4 [3680, 3010, 4122, 0, 0] [0, 0, 0, 0, 0] 
[[[1], [0]], [

([1, 163, 2868, 0, 0],
 [[[0, 0, 0, 0, 1197, 0, 0], [0, 0, 0, 0, 952, 2161, 0]],
  [[2436, 2585, 0, 0, 0, 0, 0], [3951, 3581, 4544, 0, 0, 0, 0]]])

In [231]:

def poly_mul( a , b ):
  r = [a[0].parent(0)]*(len(a)+len(b)-1)
  for i in range(len(a)):
    r[i:i+len(b)] = list_add( r[i:i+len(b)] , list_mul(b,a[i]) )
    #for j in range(len(b)):
    #  r[i+j] = r[i+j] + a[i]*b[j]
  return r

def mat_mul( mat1 , mat2 ) :
  m1_len = len(mat1[0][0])
  m2_len = len(mat2[0][0])
  r_len = m1_len+m2_len
  r00_u = poly_mul(mat1[0][0],mat2[0][0])
  r00_d = poly_mul(mat1[0][1],mat2[1][0])
  r00 = list_add(list_pad_front(r00_u,r_len),list_pad_zero(r00_d,r_len))
  r01_u = poly_mul(mat1[0][0],mat2[0][1])
  r01_d = poly_mul(mat1[0][1],mat2[1][1])
  r01 = list_add(list_pad_front(r01_u,r_len),list_pad_zero(r01_d,r_len))
  r10_u = poly_mul(mat1[1][0],mat2[0][0])
  r10_d = poly_mul(mat1[1][1],mat2[1][0])
  r10 = list_add(list_pad_front(r10_u,r_len),list_pad_zero(r10_d,r_len))
  r11_u = poly_mul(mat1[1][0],mat2[0][1])
  r11_d = poly_mul(mat1[1][1],mat2[1][1])
  r11 = list_add(list_pad_front(r11_u,r_len),list_pad_zero(r11_d,r_len))
  return [ [r00,r01] , [ r10 , r11 ] ]



def extgcd_list( a , b ):
  deg_a = a.degree()
  deg_b = b.degree()
  if deg_a < deg_b :
    a , b = b , a
    deg_a , deg_b = deg_b , deg_a
  if deg_a < 0 : return a
  if deg_a == deg_b : 
    b = b*a.leading_coefficient() - a*b.leading_coefficient()
  deg_b = deg_a - 1
  delta = deg_a - deg_b
  r1 = poly_rev( a , deg_a )
  r2 = poly_rev( b , deg_b )
  lenlist = deg_a + 1
  r2 = list_pad_zero( r2 , lenlist )
  coef_k = r1[0].parent()
  print delta , r1 , r2 , "\n"
  delta , r1 , r2 , T1 = divstep_list( delta , r1 , r2 )
  T0 = T1    # T1*Identity = T1
  print delta , r1 , r2 , '\n' , T1 , '\n'
  for i in range(deg_a+deg_b-1) :
    delta , r1 , r2 , T1 = divstep_list( delta , r1 , r2 )
    T0 = mat_mul( T1 , T0 )
    print delta , r1 , r2 , '\n' , T1 , '\n' , T0 , '\n'
  #normalize
  inv = r1[0]^-1
  r1 = map( lambda x: x*inv , r1 )
  T0[0][0] = list_mul( T0[0][0] , inv )
  T0[0][1] = list_mul( T0[0][1] , inv )
  print "output:"
  print delta , r1 , r2 , '\n' , T0 , '\n'
  return r1 , T0

In [237]:

def mat_mul_vec( mat , vec ) :
  m_l = len(mat[0][0])
  f_m00 = poly_mul(mat[0][0],vec[0])
  g_m01 = poly_mul(mat[0][1],vec[1])
  f = list_add( f_m00 , g_m01 )
  f_m10 = poly_mul(mat[1][0],vec[0])
  g_m11 = poly_mul(mat[1][1],vec[1])
  g = list_add( f_m10 , g_m11 )
  ## XXX: assert 0 for truncated part.
  rf = f[m_l-1:]
  rg = list_pad_zero( g[m_l:] , len(vec[1]) )
  return [ rf , rg ]

def extgcd_list( a , b ):
  deg_a = a.degree()
  deg_b = b.degree()
  if deg_a < deg_b :
    a , b = b , a
    deg_a , deg_b = deg_b , deg_a
  if deg_a < 0 : return a
  if deg_a == deg_b : 
    b = b*a.leading_coefficient() - a*b.leading_coefficient()
  deg_b = deg_a - 1
  delta = deg_a - deg_b
  r1 = poly_rev( a , deg_a )
  r2 = poly_rev( b , deg_b )
  lenlist = deg_a + 1
  r2 = list_pad_zero( r2 , lenlist )
  _r1 = list(r1) ###
  _r2 = list(r2) ###
  coef_k = r1[0].parent()
  print delta , r1 , r2 , "\n"
  delta , r1 , r2 , T1 = divstep_list( delta , r1 , r2 )
  T0 = T1    # T1*Identity = T1
  print delta , r1 , r2 , '\n' , T1 , '\n'
  for i in range(deg_a+deg_b-1) :
    delta , r1 , r2 , T1 = divstep_list( delta , r1 , r2 )
    T0 = deg1mat_mul( T1 , T0 )
    print delta , r1 , r2 , '\n' , T1 , '\n' , T0 , '\n'
    c = mat_mul_vec( T0 , [_r1 , _r2] ) ####
    print "chk: " , c[0] , c[1] , "\n"
  #normalize
  inv = r1[0]^-1
  r1 = map( lambda x: x*inv , r1 )
  T0[0][0] = list_mul( T0[0][0] , inv )
  T0[0][1] = list_mul( T0[0][1] , inv )
  print "output:"
  print delta , r1 , r2 , '\n' , T0 , '\n'
  return r1 , T0



In [241]:
extgcd_list( a2*b1 , b2 )

1 [3673, 1134, 3843, 701] [2898, 4092, 1754, 0] 

0 [2898, 4092, 1754, 0] [194, 2570, 2276, 0] 
[[[0], [1]], [[2898], [918]]] 

1 [2898, 4092, 1754, 0] [1653, 2630, 0, 0] 
[[[1], [0]], [[4397], [2898]]] 
[[[0, 0], [0, 1]], [[1465, 0], [2175, 4397]]] 

chk:  [2898, 4092, 1754, 0] [1653, 2630, 0, 0] 

0 [1653, 2630, 0, 0] [853, 2441, 0, 0] 
[[[0], [1]], [[1653], [1693]]] 
[[[1465, 0, 0], [2175, 4397, 0]], [[1105, 0, 0], [293, 2110, 1653]]] 

chk:  [1653, 2630, 0, 0] [853, 2441, 0, 0] 

1 [1653, 2630, 0, 0] [1093, 0, 0, 0] 
[[[1], [0]], [[3738], [1653]]] 
[[[0, 1465, 0, 0], [0, 2175, 4397, 0]], [[3938, 3698, 0, 0], [2274, 2750, 970, 0]]] 

chk:  [1653, 2630, 0, 0] [1093, 0, 0, 0] 

0 [1093, 0, 0, 0] [624, 0, 0, 0] 
[[[0], [1]], [[1093], [2938]]] 
[[[3938, 3698, 0, 0, 0], [2274, 2750, 970, 0, 0]], [[524, 2418, 3577, 0, 0], [1107, 3931, 2577, 3735, 0]]] 

chk:  [1093, 0, 0, 0] [624, 0, 0, 0] 

output:
0 [1, 0, 0, 0] [624, 0, 0, 0] 
[[[4561, 2490, 0, 0, 0], [3669, 3262, 3588, 0, 0]], [[524, 

([1, 0, 0, 0],
 [[[4561, 2490, 0, 0, 0], [3669, 3262, 3588, 0, 0]],
  [[524, 2418, 3577, 0, 0], [1107, 3931, 2577, 3735, 0]]])

In [242]:

def jumpdivstep_mat(n,delta,f,g):
  print "jumpdivstep_mat(",n,delta,f,g,")"
  assert n>0
  ### truncate 
  f,g = f[:n], g[:n]
  print "truncated(",n,"): f=",f , "  g=", g
  if n==1:
    rd,f,g,T = divstep_list( delta , f , g )
    return rd,T
  # other cases of n
  j = n//2
  print "j=",j
  delta,P1 = jumpdivstep_mat(j,delta,f,g)
  print "P1:\n" , P1 
  f,g = mat_mul_vec( P1 , (f,g) )
  print "P1x(f,g): f=" , f , "  g=", g
  delta,P2 = jumpdivstep_mat(n-j,delta,f,g)
  print "P2:\n" , P2
  #f,g = mat_mul_vec( P2 , (f,g) )
  #print "P2x(f,g): f=" , f , "  g=", g
  #f,g = kx(f).truncate(t-n+1),kx(g).truncate(t-n)
  #print "truncated: f=",f , "  g=", g
  rP = mat_mul( P2 , P1 )
  print "ret: ", delta , rP
  return delta, rP


In [248]:
f = poly_rev(a2,2)
g = poly_rev(b2,2)

In [257]:

def jumpdivsteps_list(n,delta,f,g):
  print "jumpdivsteps_list(",n,delta,f,g,")"
  assert n>0
  if n==1:
    return divstep_list( delta , f , g )
  # other cases of n
  j = n//2
  print "j=",j
  delta,P1 = jumpdivstep_mat(j,delta,f,g)
  print "P1:\n" , P1 
  f,g = mat_mul_vec( P1 , (f,g) )
  print "P1x(f,g): f=" , f , "  g=", g
  delta,P2 = jumpdivstep_mat(n-j,delta,f,g)
  print "P2:\n" , P2
  f,g = mat_mul_vec( P2 , (f,g) )
  print "P2x(f,g): f=" , f , "  g=", g
  #f,g = kx(f).truncate(t-n+1),kx(g).truncate(t-n)
  print "truncated ???: f=",f , "  g=", g
  rP = mat_mul( P2 , P1 )
  return delta, f , g , rP

In [266]:
f = poly_rev(a2*b2,4)
g = poly_rev(b2*c1,3)
g = list_pad_zero(g,len(f))
f,g

([3504, 1659, 2347, 1601, 2989], [3696, 833, 24, 3710, 0])

In [269]:

def jumpgcd_list(R0,R1):
  d = R0.degree()
  assert d>0 and d>R1.degree()
  f = poly_rev(R0,d)
  g = poly_rev(R1,R1.degree())
  g = list_pad_zero(g,len(f))
  delta,f,g,T = jumpdivsteps_list(2*d-1,1,f,g)
  return delta,f,g,T


In [270]:
jumpgcd_list( a2*b2 , b2*c1 )

jumpdivsteps_list( 7 1 [3504, 1659, 2347, 1601, 2989] [3696, 833, 24, 3710, 0] )
j= 3
jumpdivstep_mat( 3 1 [3504, 1659, 2347, 1601, 2989] [3696, 833, 24, 3710, 0] )
truncated( 3 ): f= [3504, 1659, 2347]   g= [3696, 833, 24]
j= 1
jumpdivstep_mat( 1 1 [3504, 1659, 2347] [3696, 833, 24] )
truncated( 1 ): f= [3504]   g= [3696]
P1:
[[[0], [1]], [[3696], [1087]]]
P1x(f,g): f= [3696, 833, 24]   g= [3723, 655, 0]
jumpdivstep_mat( 2 0 [3696, 833, 24] [3723, 655, 0] )
truncated( 2 ): f= [3696, 833]   g= [3723, 655]
j= 1
jumpdivstep_mat( 1 0 [3696, 833] [3723, 655] )
truncated( 1 ): f= [3696]   g= [3723]
P1:
[[[1], [0]], [[868], [3696]]]
P1x(f,g): f= [3696, 833]   g= [3680, 0]
jumpdivstep_mat( 1 1 [3696, 833] [3680, 0] )
truncated( 1 ): f= [3696]   g= [3680]
P2:
[[[0], [1]], [[3680], [895]]]
ret:  0 [[[868, 0], [3696, 0]], [[981, 3680], [2400, 0]]]
P2:
[[[868, 0], [3696, 0]], [[981, 3680], [2400, 0]]]
ret:  0 [[[2191, 0, 0], [427, 868, 0]], [[588, 0, 0], [1112, 981, 3680]]]
P1:
[[[2191, 0, 0], [4

(4,
 [3680, 3010, 4122, 0, 0],
 [0, 0, 0, 0, 0],
 [[[0, 0, 0, 0, 2191, 0, 0], [0, 0, 0, 0, 427, 868, 0]],
  [[2436, 2585, 0, 0, 0, 0, 0], [3951, 3581, 4544, 0, 0, 0, 0]]])